In [5]:
import itertools
import re
import os
import time
import random
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from time import sleep
from webdriver_manager.chrome import ChromeDriverManager


driver = webdriver.Chrome('/Users/glicia/python/chromedriver')

os.chdir('/Users/glicia/scraping/Side Effect/SEscrapings/sescraping/lists')
csv_file = pd.read_csv('k1_list.csv')

ses = []

lists = []
for index, data in csv_file.iterrows():
    data[0]
    lists.append(data[0])
    
url = 'https://www.drugs.com/sfx/'
driver.get(url)

button = driver.find_element_by_css_selector('#search > form > div > button')
serch_text = driver.find_element_by_id('livesearch-sfx')

for text_data in lists:
    wait1 = random.uniform(4,8)
    wait2 = random.uniform(1,4)
    wait3 = random.uniform(2,3)
    wait4 = random.uniform(1,3)
    text_data = (text_data + ' Side Effects')
    serch_text.send_keys(text_data)
    sleep(wait2)
    #print(wait2)
    button.click()
    sleep(wait3)
    #print(wait3)

    elem = driver.find_element_by_css_selector('#content > div.contentBox > div.ddc-media-list > div:nth-child(1) > div')
    new_url = elem.find_element_by_tag_name('a').get_attribute('href')
    res = requests.get(new_url)
    soup = BeautifulSoup(res.text, 'html.parser')
    sleep(wait4)
    #print(wait4)

    drug_name = soup.find('h1').text
    dn = drug_name.split(' Side Effects')[0]

    try:
        gntxt = soup.find('p', class_='drug-subtitle').text
        gn = gntxt.split('Generic name: ')[1]
    except:
        gn = dn

    fqecy = "N/A"

    try:
        soup.find('h2', text='What are some side effects that I need to call my doctor about right away?').name = 'new_tag'

        soup.find("h2", text="\n      What are some other side effects of this drug?\n    ").name = 'new_tag'

        ntgs = soup.find_all('new_tag')

    except:
        pass

    try:
        for el in ntgs:
            els = [i for i in itertools.takewhile(
                      lambda x: x.name not in [el.name, 'div'],
                      el.next_siblings)]
            border = soup.new_tag('border')
            el.wrap(border)
            for tag in els:
                border.append(tag)
    except:
        pass

    se_lst = []
    try:
        for j in soup.find_all('border'):
            rng_name = j.find('new_tag').text
            rng_name = rng_name.replace('\n      ', '')
            rng_name = rng_name.replace('\n    ', '')
            for se in j.find_all("li"):
                sename = se.text
                sename = sename.replace('\n      ', '')
                se_lst = [dn, gn, rng_name, fqecy, sename]
                ses.append(se_lst)
    except:
        pass

    driver.back()
    sleep(wait1)
    #print(wait1)

    button = driver.find_element_by_css_selector('#search > form > div > button')
    serch_text = driver.find_element_by_id('livesearch-sfx')

driver.quit()

df = pd.DataFrame(ses, columns=['Drug_Name', 'Generic_Name', 'Requiring_Level', 'Frequency', 'Side_Effect'])
df.to_csv('k1_220107.csv', index=False)